<a href="https://colab.research.google.com/github/genie0320/langchain/blob/main/04_RAG_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting

In [4]:
# colab 환경설정
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

from google.colab import drive
drive.mount('/content/drive')

# with open('/gdrive/My Drive/foo.txt', 'w') as f:
#   f.write('Hello Google Drive!')
# !cat '/gdrive/My Drive/foo.txt'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# from dotenv import load_dotenv
# load_dotenv()
# api_key = os.getenv("OPENAI_API_KEY")

## Load utills

In [6]:
!pip install --quiet icecream

In [7]:
import pprint
from icecream import ic

def pp(object):
  ppr = pprint.PrettyPrinter(
      # indent=40,
      width=80
      )
  return ppr.pprint(object)

function KeepClicking(){ console.log("Clicking"); document.querySelector("colab-connect-button").click() } setInterval(KeepClicking,60000)

# Model setting

In [8]:
!pip install -q langchain
!pip install -q langchain_openai

In [9]:
# ___ setting ___
LLM_MODEL = "gpt-3.5-turbo"
MAX = 50
TEMP = 1.5
# _______________

from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

openai_compbot = OpenAI(
    temperature=TEMP,
    max_tokens = MAX,
    verbose = True,
)
openai_chatbot = ChatOpenAI(
    temperature=TEMP,
    max_tokens = MAX,
    verbose = True,
)

In [10]:
ic(openai_compbot.invoke('hi'))
ic(openai_chatbot.invoke('hi'))

ic| openai_compbot.invoke('hi'): ('
                                 '
                                  '52526
                                 '
                                  '
                                 '
                                  '     
                                 '
                                  '
                                 '
                                  "Interesting Post. – washaq Jul 31 '09 at 17:50
                                 "
                                  '
                                 '
                                  '     
                                 '
                                  '
                                 '
                                  'Thanks crash31. This what flushed debate a styles fall forward as went linux '
                                  'change me tiended whenever utility only to down rattled')
ic| openai_chatbot.invoke('hi'): AIMessage(content='Hello! How can I assist you today?')


AIMessage(content='Hello! How can I assist you today?')

In [11]:
# # Comletion
# prompt = PromptTemplate.from_template( '{time} + \n\n text{name} +  {action}')
# user_input = 'Where shall we go today?'
# new_prompt = prompt.format(
#     time = "old",
#     name = "Genie",
#     action = user_input
# )
# res_llm = llm.invoke(new_prompt)

# # Chat
# template = "You are my new friend. We met {place} for {activity}."
# human_template = "{text}"
# chat_prompt = ChatPromptTemplate.from_messages([
#     ("system", template), # 값을 tuple로 전달.
#     ("human", human_template),
# ])

# prompt = chat_prompt.format_messages(place="in library", activity="being a study friend", text="Hey, Sweety!")
# res_chat = chat_model.invoke(prompt).content

In [12]:
# print(prompt)
# print(res_chat)

# RAG setting

## Loader

- [ ] 웹페이지 로딩
- [ ] 옵시디언 로딩
- [ ] 노션 로딩
- [ ] `from langchain_community.document_loaders import UnstructuredPDFLoader`

In [13]:
SOURCE_folder = "/content/source"
SOURCE = "/content/source/1-1그로스 해킹, 마케팅과 어떻게 다른가요_ - PUBLY.pdf"

In [14]:
# !pip install -q langchain_community
# !pip install -q pypdf

### 파일불러오기

#### 폴더째로 불러오기

- [ ] `documents = loader.load()`, `pages = loader.load_and_split()` 의 차이는?

In [15]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
loader = DirectoryLoader(SOURCE_folder, glob='**/*.pdf', show_progress=True, loader_cls=PyPDFLoader)
# documents = loader.load()
# print(len(documents), '\n\n', documents[0])
pages = loader.load_and_split()
print(len(pages), '\n\n', pages[0])

# 위의 둘은 결과적으로 똑같다. 그러나 아래것이 조금 더 빠르다.

100%|██████████| 8/8 [00:16<00:00,  2.01s/it]

71 

 page_content="24. 2. 9. 오후  9:49 가설 검증은  비즈니스의  기본이다 : 조금  잃고  많이  따는  법  - PUBLY\nhttps://publy.co/content/4608?fr=set-bottom-list 1/13가설 검증은 비즈니스의 기본이다: 조금 잃고\n많이 따는 법\n저자김민우\n마케팅 트렌드/실무서비스 기획/데이터/UX\n5,864명이 봤어요 • 91% 만족\n과학시간도 아니고 웬 가설?\nEditor's Comment\n이번 웹북의 제목 '리스크를 줄이는 그로스해킹 전략: 퍼블리는 A/B 테스트를 어떻게 하고 있나'은 퍼블리 고객분들을\n대상으로 설문조사를 해 정해진 제목입니다. 퍼블리는 앞으로도 고객 여러분들의 의견을 적극 반영해 콘텐츠에 만족하실\n수 있도록 노력하겠습니다.\n저자 김민우\n그로스 컨설턴트, (전)퍼블리 VP of Growth > 프로필 더 보기\n가설 수립과 가설 검증. 아마 학창 시절 과학 시간 이후로 들을 일도 말할 일도 없었던 개념들일\n것입니다. 벌써 학교를 졸업한 우리가 왜 가설에 대해 알아야 할까요? 그 이유는 그로스의 핵심,\n더 본질적으로는 스타트업의 핵심이 가설을 수립하고, 검증하며 배움을 얻는 것이기 때문입니다.\n \n여러분은 스타트업이 무엇이라고 생각하시나요? 그리고 스타트업에서 일하는 것에는 어떤\n의미가 있다고 생각하시나요? 스타트업이 무엇인지, 그 정의를 이해하면 가설 수립과 검증의\n중요성 역시 이해할 수 있습니다.\n \n스타트업이 무엇인지 사람들에게 묻는다면, 일반적으로 다음과 같은 대답이 나올 것입니다.\n새로운 기술로 혁신적인 사업을 하는 회사\n젊은 사람들이 일하는 회사\n자유로운 분위기를 가진 회사\n새로 시작해서 규모가 작은 회사\n그러나 저는 이런 요소들이 스타트업 중 일부가 가진 속성일 수는 있어도, 스타트업을 규정하는\n본질적 요소는 아니라고 생각합니다.\n \n스타트업을 위한 경영학이라 할 수 있는 고객 개발(customer developm

#### 파일 하나 불러오기

In [16]:
# 이건 PDF
# from langchain_community.document_loaders import PyPDFLoader
# loader = PyPDFLoader(SOURCE)
# pages = loader.load_and_split()
# print(len(pages), '\n\n', pages[0])

In [17]:
# 이건 뭔...지
# with open("../../state_of_the_union.txt") as f: # 이건 로컬파일일 때...이야기인가. loader와 무슨 관계?
#     state_of_the_union = f.read()

### WORD 파일

In [18]:
# Docx2TextLoader
# CSVLoader : 이건 좀 사용법이 다르다.


### 웹페이지 불러오기

In [19]:
# TARGET_URL = 'https://blog.ab180.co/posts/growth-hacking-5-features'
# TARGET_URLS = [
#     'https://blog.ab180.co/posts/growth-hacking-5-features',
#     'https://dbr.donga.com/article/view/1202/article_no/9026/ac/magazine',
#     'https://blog.hsad.co.kr/3323',
#     'https://groobee.net/blog/%eb%b9%84%ec%a6%88%eb%8b%88%ec%8a%a4%eb%a5%bc-%ec%9c%84%ed%95%9c-%ed%95%b5%ec%8b%ac-%ec%a0%84%eb%9e%b5-%ea%b7%b8%eb%a1%9c%ec%8a%a4-%eb%a7%88%ec%bc%80%ed%8c%85%ec%9d%98-%eb%aa%a8%eb%93%a0-%ea%b2%83/',
#     'https://groobee.net/blog/%EB%B9%84%EC%A6%88%EB%8B%88%EC%8A%A4%EB%A5%BC-%EC%9C%84%ED%95%9C-%ED%95%B5%EC%8B%AC-%EC%A0%84%EB%9E%B5-%EA%B7%B8%EB%A1%9C%EC%8A%A4-%ED%95%B4%ED%82%B9%EC%9D%98-%EB%AA%A8%EB%93%A0-%EA%B2%83-2/',
#     'https://groobee.net/blog/%eb%b9%84%ec%a6%88%eb%8b%88%ec%8a%a4%eb%a5%bc-%ec%9c%84%ed%95%9c-%ed%95%b5%ec%8b%ac-%ec%a0%84%eb%9e%b5-%ea%b7%b8%eb%a1%9c%ec%8a%a4-%ed%95%b4%ed%82%b9%ec%9d%98-%eb%aa%a8%eb%93%a0-%ea%b2%83-3/',
#     'https://groobee.net/blog/%eb%b9%84%ec%a6%88%eb%8b%88%ec%8a%a4%eb%a5%bc-%ec%9c%84%ed%95%9c-%ed%95%b5%ec%8b%ac-%ec%a0%84%eb%9e%b5-%ea%b7%b8%eb%a1%9c%ec%8a%a4-%ed%95%b4%ed%82%b9%ec%9d%98-%eb%aa%a8%eb%93%a0-%ea%b2%83-4/',
#     'https://groobee.net/blog/%eb%b9%84%ec%a6%88%eb%8b%88%ec%8a%a4%eb%a5%bc-%ec%9c%84%ed%95%9c-%ed%95%b5%ec%8b%ac-%ec%a0%84%eb%9e%b5-%ea%b7%b8%eb%a1%9c%ec%8a%a4-%ed%95%b4%ed%82%b9%ec%9d%98-%eb%aa%a8%eb%93%a0-%ea%b2%83-5/',
#     'https://groobee.net/blog/%EB%B9%84%EC%A6%88%EB%8B%88%EC%8A%A4%EB%A5%BC-%EC%9C%84%ED%95%9C-%ED%95%B5%EC%8B%AC-%EC%A0%84%EB%9E%B5-%EA%B7%B8%EB%A1%9C%EC%8A%A4-%ED%95%B4%ED%82%B9%EC%9D%98-%EB%AA%A8%EB%93%A0-%EA%B2%83-6/'
# ]

#### 한페이지 불러오기

In [20]:
# from langchain.document_loaders import WebBaseLoader
# loader = WebBaseLoader(TARGET_URL)

# web_data = loader.load()
# print(len(web_data), '\n', web_data[0])

#### 여러페이지 불러오기

한번에 많이 가져올 수 있어서 좋으나... 결과물이 좀 더럽다.

In [21]:
# !pip install -q unstructured

In [22]:
# from langchain.document_loaders import UnstructuredURLLoader
# loader = UnstructuredURLLoader(urls = TARGET_URLS)
# web_datas = loader.load()

In [23]:
# print(len(web_datas), '\n', web_datas[0])

### 데이터 모양 살펴보기

In [24]:
# for p in pages[0:10]:
#     with open('/content/drive/MyDrive/bar_split.txt', 'a') as f:
#         f.write(p.page_content)
# # # !cat '/gdrive/My Drive/foo.txt'

### 줄바꿈등 제거하기

In [25]:
# TODO: 줄바꿈 등을 제거한 후

## Splitter

- [ ] `# from langchain_experimental.text_splitter import SemanticChunker > 이건 openai의 실험적 기능이라는데... 한번 써보고 싶음.`

In [26]:
# !pip install -q langchain_core langchain_community
# !pip install -q sentence-transformers

### 글자수로 맞춰 분할.

In [28]:
# loader : pages

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = '\n',
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function= len # 이 경우, 단순히 글자수를 기준으로 1000을 세라는 것.
)

chunks = text_splitter.split_documents(pages) # 얘는 배열 그대로를 처리할 수 있다.
# texts= text_splitter.split_text(pages) # 얘는 str이어야 함.
chunks[0]

Document(page_content="24. 2. 9. 오후  9:49 가설 검증은  비즈니스의  기본이다 : 조금  잃고  많이  따는  법  - PUBLY\nhttps://publy.co/content/4608?fr=set-bottom-list 1/13가설 검증은 비즈니스의 기본이다: 조금 잃고\n많이 따는 법\n저자김민우\n마케팅 트렌드/실무서비스 기획/데이터/UX\n5,864명이 봤어요 • 91% 만족\n과학시간도 아니고 웬 가설?\nEditor's Comment\n이번 웹북의 제목 '리스크를 줄이는 그로스해킹 전략: 퍼블리는 A/B 테스트를 어떻게 하고 있나'은 퍼블리 고객분들을\n대상으로 설문조사를 해 정해진 제목입니다. 퍼블리는 앞으로도 고객 여러분들의 의견을 적극 반영해 콘텐츠에 만족하실\n수 있도록 노력하겠습니다.\n저자 김민우\n그로스 컨설턴트, (전)퍼블리 VP of Growth > 프로필 더 보기\n가설 수립과 가설 검증. 아마 학창 시절 과학 시간 이후로 들을 일도 말할 일도 없었던 개념들일\n것입니다. 벌써 학교를 졸업한 우리가 왜 가설에 대해 알아야 할까요? 그 이유는 그로스의 핵심,\n더 본질적으로는 스타트업의 핵심이 가설을 수립하고, 검증하며 배움을 얻는 것이기 때문입니다.\n \n여러분은 스타트업이 무엇이라고 생각하시나요? 그리고 스타트업에서 일하는 것에는 어떤\n의미가 있다고 생각하시나요? 스타트업이 무엇인지, 그 정의를 이해하면 가설 수립과 검증의\n중요성 역시 이해할 수 있습니다.\n \n스타트업이 무엇인지 사람들에게 묻는다면, 일반적으로 다음과 같은 대답이 나올 것입니다.\n새로운 기술로 혁신적인 사업을 하는 회사\n젊은 사람들이 일하는 회사\n자유로운 분위기를 가진 회사\n새로 시작해서 규모가 작은 회사\n그러나 저는 이런 요소들이 스타트업 중 일부가 가진 속성일 수는 있어도, 스타트업을 규정하는\n본질적 요소는 아니라고 생각합니다.\n \n스타트업을 위한 경영학이라 할 수 있는 고객 개발(customer devel

In [29]:
len(chunks)

129

In [30]:
lens = []
for n in chunks:
    lens.append(len(n.page_content))
print(sorted(lens, reverse=True))

[1000, 1000, 1000, 1000, 999, 998, 998, 997, 997, 994, 992, 992, 991, 991, 989, 989, 989, 988, 988, 987, 986, 986, 984, 984, 983, 983, 983, 982, 982, 982, 981, 980, 979, 978, 977, 975, 975, 975, 974, 974, 973, 972, 970, 969, 969, 968, 968, 965, 965, 965, 965, 962, 960, 960, 957, 954, 952, 950, 947, 938, 932, 927, 905, 898, 892, 866, 864, 859, 843, 834, 827, 820, 816, 811, 801, 776, 756, 730, 702, 702, 701, 663, 659, 651, 646, 645, 636, 624, 621, 603, 601, 594, 575, 573, 567, 559, 538, 526, 521, 504, 502, 491, 483, 470, 416, 408, 404, 401, 401, 396, 395, 368, 359, 358, 330, 318, 288, 268, 249, 234, 226, 192, 166, 156, 156, 151, 142, 120, 97]


### 재귀적으로 분할... 그러나....

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 적정한 길이로 잘라주고
rc_splitter = RecursiveCharacterTextSplitter(
    # separator="\n\n", : 이 옵션은 의미가 없다. 알아서 맞춰서 잘라준다.
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
)

docs = []
for document in pages:
    docs.append(document.page_content)

# texts = splitter.split_documents(pages) # 단순 커터
rc_chunks = rc_splitter.create_documents(docs) # 옵션을 줄 수 있다. []을 받는다.


### Token 길이에 맞춰 분할

In [32]:
# !pip install -q tiktoken

In [33]:
# token 단위로 분할하는 기능
import tiktoken
tokenizer = tiktoken.get_encoding('cl100k_base') # openai에서 사용하는 규약이라고 함.

# 해당 텍스트를 처리하기 위한 token의 수를 세는 함수
def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

tiktoken_len(pages[0].page_content)

911

In [42]:
# rc_t_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder( > 이런 것도 있다던데.
rc_t_splitter = RecursiveCharacterTextSplitter(
    is_separator_regex=False,
    chunk_size=1000,
    chunk_overlap=100,
    length_function=tiktoken_len,# 얘는 토큰길이에 따름.
)

In [43]:
tk_chunks = rc_t_splitter.split_documents(pages)
len(tk_chunks)

119

In [46]:
tk_chunks[0]

Document(page_content="24. 2. 9. 오후  9:49 가설 검증은  비즈니스의  기본이다 : 조금  잃고  많이  따는  법  - PUBLY\nhttps://publy.co/content/4608?fr=set-bottom-list 1/13가설 검증은 비즈니스의 기본이다: 조금 잃고\n많이 따는 법\n저자김민우\n마케팅 트렌드/실무서비스 기획/데이터/UX\n5,864명이 봤어요 • 91% 만족\n과학시간도 아니고 웬 가설?\nEditor's Comment\n이번 웹북의 제목 '리스크를 줄이는 그로스해킹 전략: 퍼블리는 A/B 테스트를 어떻게 하고 있나'은 퍼블리 고객분들을\n대상으로 설문조사를 해 정해진 제목입니다. 퍼블리는 앞으로도 고객 여러분들의 의견을 적극 반영해 콘텐츠에 만족하실\n수 있도록 노력하겠습니다.\n저자 김민우\n그로스 컨설턴트, (전)퍼블리 VP of Growth > 프로필 더 보기\n가설 수립과 가설 검증. 아마 학창 시절 과학 시간 이후로 들을 일도 말할 일도 없었던 개념들일\n것입니다. 벌써 학교를 졸업한 우리가 왜 가설에 대해 알아야 할까요? 그 이유는 그로스의 핵심,\n더 본질적으로는 스타트업의 핵심이 가설을 수립하고, 검증하며 배움을 얻는 것이기 때문입니다.\n \n여러분은 스타트업이 무엇이라고 생각하시나요? 그리고 스타트업에서 일하는 것에는 어떤\n의미가 있다고 생각하시나요? 스타트업이 무엇인지, 그 정의를 이해하면 가설 수립과 검증의\n중요성 역시 이해할 수 있습니다.\n \n스타트업이 무엇인지 사람들에게 묻는다면, 일반적으로 다음과 같은 대답이 나올 것입니다.\n새로운 기술로 혁신적인 사업을 하는 회사\n젊은 사람들이 일하는 회사\n자유로운 분위기를 가진 회사\n새로 시작해서 규모가 작은 회사\n그러나 저는 이런 요소들이 스타트업 중 일부가 가진 속성일 수는 있어도, 스타트업을 규정하는\n본질적 요소는 아니라고 생각합니다.\n \n스타트업을 위한 경영학이라 할 수 있는 고객 개발(customer devel

In [44]:
tk_lens = []
for n in tk_chunks:
    tk_lens.append(len(n.page_content))
print(sorted(tk_lens, reverse=True))

[1223, 1208, 1179, 1178, 1166, 1165, 1161, 1161, 1154, 1151, 1142, 1139, 1137, 1135, 1127, 1124, 1121, 1119, 1119, 1115, 1113, 1112, 1099, 1098, 1094, 1094, 1093, 1090, 1086, 1086, 1083, 1081, 1080, 1080, 1079, 1077, 1073, 1072, 1071, 1071, 1064, 1058, 1056, 1053, 1052, 1051, 1050, 1048, 1047, 1045, 1043, 1042, 1037, 1034, 1027, 1004, 1000, 987, 965, 938, 913, 905, 892, 864, 859, 827, 790, 784, 783, 758, 756, 754, 725, 712, 702, 694, 663, 653, 646, 640, 623, 606, 603, 597, 544, 538, 532, 521, 519, 517, 513, 512, 506, 504, 502, 499, 478, 467, 459, 425, 408, 399, 368, 358, 352, 330, 317, 302, 301, 279, 263, 259, 240, 208, 176, 176, 163, 117, 99]


## Embedding / Saving

In [38]:
DB_URL = 'store/openai/publy'

In [39]:
from langchain_community.vectorstores import Chroma
# chroma_client = chromadb.Client() > 이건 뭐에 필요한 것인지...

In [40]:
# from langchain_openai import OpenAIEmbeddings
# openai_embed = OpenAIEmbeddings()
# vector_stores = Chroma.from_documents(texts, openai_embed)
# vector_stores = Chroma.from_documents(
#     texts,
#     openai_embed,
#     persist_directory = DB_URL
# )
# print('openai_embed : Document Stored!!')

In [41]:
print(openai_embed)

NameError: name 'openai_embed' is not defined

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
hug_embed = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# hug_embed = HuggingFaceEmbeddings(model_name="BlinkDL/rwkv-5-world")
# vector_stores = Chroma.from_documents(texts, hug_embed)
vector_stores = Chroma.from_documents(
    texts,
    hug_embed,
    persist_directory = DB_URL
)
print('hug_embed : Document Stored!!')

In [ ]:
print(hug_embed)

---------------------------------

## Retrieval

In [ ]:
# 그리고 retreiver를 만들어서
from langchain.chains import RetrievalQA

from_db = Chroma(persist_directory=DB_URL, embedding_function=embedding)
retriever = from_db.as_retriever(search_type = 'similarity', search_kwargs={'k' :2})
print(docs[0].page_content)

In [ ]:
## Retrieve from DB
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
load_vector_store = Chroma(
    persist_directory = DIRECT,
    embedding_function = embeddings,
    )

In [ ]:
retriever = load_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
user_input_embedding = input('검색할 키워드 입력 :')
query = user_input_embedding
docs = db.similarity_search(query)
print(docs[0].page_content)

In [ ]:
# user_input_multiquery = input('검색할 키워드 입력 :')
# question = user_input_multiquery
# llm = chat_model

# retriever_from_llm = MultiQueryRetriever.from_llm(
#     retriever=db.as_retriever(), llm=llm
# )
# print(retriever_from_llm)
# print(len(res))
# res = retriever_from_llm.get_relevant_documents(query = question)

In [ ]:
query = '팀장과 팀원의 차이는?'

In [ ]:
docs = retriever.get_relevant_documents(query)
print(len(docs), docs[0])

In [ ]:
context = []
for d in docs:
    context.append(d.page_content)
context

## Making QA chain

In [ ]:
from langchain.chains.question_answering import load_qa_chain
# RAG 체인을 만들고
qa = RetrievalQA.from_chain_type(
    llm=ask_openai(), chain_type='stuff', retriever = retriever, return_source_documents=True
)

# 질문을 하면 된다.
rerult = qa({'query': user_input})

# print results
rerult

## Generate Answer

In [ ]:
## Make answer with LLM
from langchain.chains import RetrievalQA

user_input_multiquery = input('검색할 키워드 입력 :')
question = user_input_multiquery
llm = chat_model
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())
result = qa_chain.invoke({'query':question})
# print(result)
print(result['result'])

In [ ]:
from langchain_community.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

In [ ]:
prompt_template = PromptTemplate.from_template('''
Use following context to answer the user's question in Korean.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
---------------------
Context :{context_str}
---------------------

Question: {query_str}

Only return the helpful answer below and nothing else.
Answer:
''')

In [ ]:
question = "팀장과 팀원의 차이는?"

In [ ]:
# prompt = prompt_template.format(
#     context_str = context,
#     query_str = question
#     )

In [ ]:
repo_id = "google/flan-t5-xxl"

llm = HuggingFaceHub(
    huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN,
    repo_id = repo_id,
    model_kwargs = {'temperature':0.3, "max_length" : 500}
)

In [ ]:
!pip install langchainhub

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

In [ ]:
import pprint

In [ ]:
# pprint(prompt)
prompt

In [ ]:
chain_type_kwargs = {"prompt": prompt}
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type='stuff',
    retriever = retriever,
    return_source_documents = True,
    chain_type_kwargs = chain_type_kwargs,
    verbose = True
)

# qa_chain = RetrievalQA.from_chain_type(
#     llm,
#     retriever=retriever,
#     chain_type_kwargs={"prompt": prompt},
#     verbose = True
# )

question = "팀장과 팀원의 차이는?"
result = qa_chain({"query": question})
result["result"]

## 이 모든 것을 한번에 해결해주는... VectorstoreIndexCreator

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator(
    # 텍스트를 자르고+
    text_splitter=splitter,
    # 임베딩처리를 하고
    embedding = embedding,
    # Chroma에 로딩한다.
    vectorstore_cls=Chroma
).from_loaders([loader])

In [ ]:
index.query(
    llm = ask_openai(),
    question = user_input,
    chain_type = 'stuff',
    vectorstore_kwargs = retriever
    )

------------------------------------

# 꽁수

## Saving data to Google Drive

* [PyDrive reference](https://pythonhosted.org/PyDrive/)
* [Google Drive API reference](https://developers.google.com/drive/v3/reference/)

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a text file.
uploaded = drive.CreateFile({'title': 'Sample file.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# After executing the cell above, a new file named 'Sample file.txt' will appear in your [drive.google.com](https://drive.google.com/) file list.

## Listing files in Google Drive

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "title contains '.txt' and 'root' in parents"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

## Downloading files or importing data from Google Drive

* [PyDrive reference](https://pythonhosted.org/PyDrive/)
* [Google Drive API reference](https://developers.google.com/drive/v3/reference/)

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = 'REPLACE_WITH_YOUR_FILE_ID'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

## Downloading files to your local file system

`files.download` will invoke a browser download of the file to your local computer.

In [ ]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

------------------------------------------

# 더 공부할 것

- chain 의 종류 stuff 이외에?
chain의 옵션들. return_intermediate_steps?

        load_qa_chain  
        RetrievalQA  
        VectorstoreIndexCreator  
        ConversationalRetrievalChain  

- chatmemory에 대한 정리

- document loader option 정리

- local coopilot with lm studio.
https://www.toolify.ai/ai-news/build-your-own-coding-copilot-in-vs-code-with-colab-507366

- colab with llama.cpp HF
https://shiv248.medium.com/hosting-open-source-llm-model-on-google-colaboratory-cc14a42d4053

## 랭체인에서 chain(), chain.run(), chain,invoke()의 차이.

- chain() 과 chain.run()은 거의 같은 기능을 했으나, 지금은 deprecate됨.
- 현재는 chain.invoke()를 사용한다. 기능적으로는 별로 달라진거 없다.

# 삽질의 추억

## 시도1 : 한번에 자료문서를 다 때려 넣으면,
- 일단 비싸고 모델에 따라서는 처리가 불가능한 경우가 발생한다.

In [ ]:
# chain = load_qa_chain(llm = ask_openai(), chain_type='stuff', verbose=True)
# user_input = '아내가 먹고 싶은 것은?'
# chain.run(input_documents=docs, question=user_input)

'''
BadRequestError: Error code: 400 - {
    'error': {
      'message': "This model's maximum context length is 4097 tokens,
      however you requested 11185 tokens (10929 in your prompt; 256 for the completion).
      Please reduce your prompt; or completion length.",
      'type': 'invalid_request_error',
      'param': None, 'code': None
      }
      }
'''